In [1]:
##DESCRIPTION

project_id = "1628ba4d-8900-41a6-88a3-2c0868b53566"


In [2]:
install.packages("RColorBrewer", repos='http://cran.us.r-project.org')
install.packages("gridExtra")
install.packages("getPass")
install.packages("RPostgreSQL")

library("RColorBrewer")
library(ggplot2)
library(gridExtra)
library(getPass)
library(RPostgreSQL)


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Loading required package: DBI



In [3]:
db_connection <- DBI::dbConnect(dbDriver(drvName = "PostgreSQL"), dbname = Sys.getenv("DB_NAME"), host=Sys.getenv("HOST_NAME"), port="5432", user=Sys.getenv("POSTGRES_USERNAME"), password=Sys.getenv("POSTGRES_PASSWORD"))  

sql_operational_profile = "
    SELECT users, frequency FROM operational_profile_observations 
        WHERE operational_profile = (SELECT id FROM operational_profiles WHERE project = ?project)"
operational_profile <- dbGetQuery(db_connection, sqlInterpolate(db_connection, sql_operational_profile, project = project_id))



In [4]:
#raw_data_content = readLines("experiments_0.5_and_1GB_benchflow_output.temp.csv")
#raw_data <- read.csv(textConnection(raw_data_content), header=TRUE, sep=";", dec = ",")

#raw_data <- raw_data[,-2]

sql_all_data = "
    SELECT tests.id::text AS test_id, test_sets.id::text AS test_set_id, test_properties.value::numeric AS users, metrics.abbreviation AS metric, items.name AS item_name, results.value AS item_value
        FROM results 
        INNER JOIN tests ON results.test = tests.id
        INNER JOIN items ON results.item = items.id
        INNER JOIN test_properties ON (test_properties.test = tests.id AND test_properties.name = 'no_of_users')
        INNER JOIN metrics ON results.metric = metrics.id 
        INNER JOIN test_set_tests ON (test_set_tests.test = tests.id)
        INNER JOIN test_sets ON (test_sets.id = test_set_tests.test_set AND test_sets.project = tests.project)
        WHERE tests.project = ?project"
all_data = dbGetQuery(db_connection, sqlInterpolate(db_connection, sql_all_data, project = project_id))

list_of_microservices = as.data.frame(unique(all_data[,5]))
no_of_microservices = nrow(list_of_microservices)

test_users_metric<-unique(all_data[,c(1:4)])
test_users_metric[list_of_microservices[,1]]<-NA

for (i in 1:nrow(test_users_metric)) {
    search_test_id <- test_users_metric[i,1]
    search_metric <- test_users_metric[i,4]
    
    for (j in 1:no_of_microservices) {
        search_microservice <- list_of_microservices[j,]
        
        row <- which(all_data$test_id == search_test_id & all_data$metric == search_metric & all_data$item_name == search_microservice)
        found_value = all_data$item_value[row]
        test_users_metric[i,j+4] <- found_value
    }
}
raw_data <- test_users_metric

tests <- unique(raw_data[,1:3])

max_no_of_users <- max(raw_data[,3])
min_no_of_users <- min(raw_data[,3])

user_load <- operational_profile[,1]
access_count <- operational_profile[,2]
max_no_of_requests <- max(user_load)
scale_factor <- max_no_of_users/max_no_of_requests
scaled_user_load <- floor(scale_factor * user_load)

In [5]:
##Create aggregate values (by fifty) of the user frequency from "operational_profile" 

calculate_aggregated_values <- function() {    
    access_frequency <- access_count/sum(access_count)
    by_fifty <- which(scaled_user_load %% 50 == 0)
    no_of_aggregated_rows = length(by_fifty)

    binProb <- c()
    for (i in 1:no_of_aggregated_rows) {
        if (i==1) {
            binProb[i] <- sum(access_frequency[1:by_fifty[i]])
        } else {
            binProb[i] <- sum(access_frequency[(by_fifty[i-1]+1):by_fifty[i]])
        }
    }

    matrix(c(scaled_user_load[by_fifty], binProb), ncol=2, nrow=no_of_aggregated_rows, dimnames=list(c(1:no_of_aggregated_rows), c("Workload (number of users)", "Domain metric per workload")))
}

aggregated_values_from_operational_profile <- calculate_aggregated_values()
aggregated_values_from_operational_profile

,Workload (number of users),Domain metric per workload
1,50,0.10582011
2,100,0.18518519
3,150,0.22222222
4,200,0.22222222
5,250,0.20370370
6,300,0.06084656


In [6]:
#Define the threshold for each service. The threshold is a vector computed as avg+3*SD for the configuration with 
#Users=2, Memory=4, CPU=1, CartReplica=1   

calculate_threshold <- function() {
    data_of_min_user<-raw_data[raw_data$users==min_no_of_users,]
    test_of_min_user<-tests[tests$users==min_no_of_users,]

    avg <-data_of_min_user[data_of_min_user$metric=="art",][,-c(1:4)]
    sd <- data_of_min_user[data_of_min_user$metric=="sdrt",][,-c(1:4)]
    data.frame(test_of_min_user,avg+3*sd)
}

threshold<-calculate_threshold()

#Check the first line of the dataframe thereshold: it must be one line
head(threshold)

,test_id,test_set_id,users,createOrder,basket,getCatalogue,getItem,getCart,login,getOrders,⋯,tags,getCustomer,viewOrdersPage,cataloguePage,getRelated,addToCart,catalogueSize,getAddress,getCard,showDetails
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3877,df6da4bc-1c84-464f-8841-e491df175d82,c01ac2ac-53ea-44c3-9e0a-02ae7ce459f5,2,0.03,0.002,0.008,0.027,0.074,0.089,0.026,⋯,0.016,0.027,0.005,0.008,0.034,0.005,0.024,0.034,0.008,0.002


In [7]:
#Exclude case with user = 2 from dataFile and check whether each service passes or fail: avg<threshol (Pass). 
#Compute the relative mass for each configuration

tests_without_benchmark<-tests[!tests$users==min_no_of_users,]
raw_data_without_benchmark<-raw_data[!raw_data$users==min_no_of_users,]

avg<-raw_data_without_benchmark[raw_data_without_benchmark$metric=="art",-4]
sd<-raw_data_without_benchmark[raw_data_without_benchmark$metric=="sdrt",-4]
mix<-raw_data_without_benchmark[raw_data_without_benchmark$metric=="mix",-4]

#Check pass/fail for each service. the "mix" value is 0 if fail and mixTemp if pass. Compute the relative mass for each configuration
pass_criteria<-avg

calculate_relative_mass <- function() {    
    relative_mass<-c()

    mix_of_passing_tests<-as.data.frame(matrix(nrow=nrow(tests_without_benchmark), ncol=ncol(raw_data_without_benchmark)-1))

    for(j in 1:nrow(pass_criteria)){
        mix_of_passing_tests[j,]<-mix[j,]
        for(i in 3:(2+no_of_microservices)){
            if(pass_criteria[j,i]>threshold[i]){mix_of_passing_tests[j,i]<-0}
        }
        relative_mass[j]<-sum(mix_of_passing_tests[j,3:(2+no_of_microservices)])
    }
   
    relative_mass
}

relative_mass <- calculate_relative_mass()

#Show first lines of passCriteria
head(pass_criteria)

,test_id,test_set_id,users,createOrder,basket,getCatalogue,getItem,getCart,login,getOrders,⋯,tags,getCustomer,viewOrdersPage,cataloguePage,getRelated,addToCart,catalogueSize,getAddress,getCard,showDetails
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,80891087-7e5c-4134-b547-1fb73828b73f,ed62e924-cb3c-406b-8e89-1009eedc5b1c,150,0.011,0.002,0.063,0.063,0.044,0.072,0.011,⋯,0.061,0.028,0.002,0.060,0.064,0.003,0.062,0.024,0.021,0.002
58,b890345a-c3c7-4ecc-a398-15878a5e9eac,1d87c9cb-58e4-46ba-9b7f-1511ce637f52,300,0.008,0.002,0.014,0.017,16.504,9.274,0.008,⋯,0.014,0.009,0.001,0.014,0.017,0.003,0.014,0.012,0.007,0.002
115,92865375-57e8-4dbe-b6db-8639595abc91,ed62e924-cb3c-406b-8e89-1009eedc5b1c,250,0.056,0.002,0.432,0.426,0.317,0.530,0.058,⋯,0.439,0.207,0.002,0.423,0.428,0.003,0.427,0.207,0.208,0.002
172,e363a12e-cf13-4971-bddf-f03159234bb4,567f4fa3-44e0-4618-9940-2c2913b2bed5,50,0.007,0.001,0.004,0.005,0.009,0.016,0.008,⋯,0.005,0.005,0.001,0.004,0.005,0.003,0.005,0.005,0.005,0.001
229,d824a86f-4138-4142-8815-d326de94f4bc,6492264c-060c-4933-93e8-0004ec63294a,300,0.053,0.002,0.414,0.417,0.512,0.693,0.056,⋯,0.408,0.202,0.002,0.408,0.413,0.003,0.418,0.199,0.208,0.002
286,e3d5ed4e-d9c5-465b-a3ba-aa8253f51aef,ed62e924-cb3c-406b-8e89-1009eedc5b1c,100,0.007,0.002,0.004,0.005,0.007,0.014,0.007,⋯,0.005,0.005,0.001,0.004,0.005,0.003,0.005,0.005,0.005,0.001


In [8]:
#Compute the domain metric for each configuration
tests_without_benchmark$relative_mass<-relative_mass

absolute_mass<-c()
for(j in 1:nrow(tests_without_benchmark)) {
    absolute_mass[j]<-tests_without_benchmark[j,"relative_mass"]*aggregated_values_from_operational_profile[match(tests_without_benchmark[j,"users"], aggregated_values_from_operational_profile[,1]),2]
}
tests_without_benchmark$absolute_mass<-absolute_mass

test_sets<-as.data.frame(unique(all_data[,2]))
colnames(test_sets)[1] <- "test_set_id"

set<-list()
domain_metric_list<-list()
for(i in 1:nrow(test_sets)){
    set[[i]]<-tests_without_benchmark[which(tests_without_benchmark[,2] == test_sets[i,1]),]
    domain_metric_list[[i]]<-set[[i]][,c(3,5)][order(set[[i]][,c(3,5)][,1]),]
}

#Uncomment this to show first lines of domain_metric_list
#head(domain_metric_list)
domain_metric_list

,users,absolute_mass
,<dbl>,<dbl>
1540,50,0.10118519
286,100,0.17720370
1,150,0.10820000
2053,200,0.04051111
115,250,0.03723704
343,300,0.01108624
,users,absolute_mass
,<dbl>,<dbl>
1483,50,0.10128042


In [9]:
#Compute Cumulative Domain metric: summing up absoluteMass over users for each configuration
test_sets$domain_metric<-0
for(i in 1:nrow(test_sets)){
    test_sets[i,2]<-round(sum(tests_without_benchmark[which(tests_without_benchmark[,2] == test_sets[i,1]),"absolute_mass"]),4)
}
domain_metric<-test_sets

domain_metric

test_set_id,domain_metric
<chr>,<dbl>
ed62e924-cb3c-406b-8e89-1009eedc5b1c,0.4754
1d87c9cb-58e4-46ba-9b7f-1511ce637f52,0.5839
567f4fa3-44e0-4618-9940-2c2913b2bed5,0.4753
6492264c-060c-4933-93e8-0004ec63294a,0.6996
4d86eda1-c7d7-41f7-84d1-c8f3f363b9d2,0.4720
be1ce813-c312-40ae-ae89-ca94d9b79567,0.4989
f76ddf6c-0950-42ba-821a-918a002cf3ff,0.4755
6275444c-6031-4164-ba33-f934d70ceb87,0.6220
3074578d-738d-46ba-bb94-e83332a27abf,0.5007


In [24]:
#Plot operational_profile against domain metric for each configuration

plot(aggregated_values_from_operational_profile, xlim=c(50, max_no_of_users), ylim=c(0, 0.3),cex.lab=1.3)
polygon(c(50,aggregated_values_from_operational_profile[,1],max_no_of_users),c(0,aggregated_values_from_operational_profile[,2],0), col="brown", lty = 1, lwd = 2, border = "black")
color=heat.colors(11)
color_transparent <- adjustcolor(color, alpha.f = 0.2) 

sorted_domain_metric<-domain_metric
k<-which(sorted_domain_metric[,2]==max(sorted_domain_metric[,2]))
#Green line whithin the polygon is the best domain matric line. 
#It corresponds to the second line in the final table below

for(i in 1:nrow(test_sets)) {
    lines(domain_metric_list[[i]], type="l", col=heat.colors(11)[i])    
    lines(domain_metric_list[[k]], type="l", col="green")
    polygon(c(50,t(domain_metric_list[[i]][1]),max_no_of_users),c(0,t(domain_metric_list[[i]][2]),0), col=color_transparent[i], lty = 1, lwd = 1 , border = rainbow(11)[i])
}

text(aggregated_values_from_operational_profile,labels = round(aggregated_values_from_operational_profile[1:6,2],3), pos=3, col="black")

graphics.off()

In [11]:
DBI::dbDisconnect(db_connection)

[1] TRUE